In [21]:
import math as math
import numpy as np
from cv2 import imread, imwrite, line as l
from Utilities import sharpen, blur, edgeDetector

def displayImage(img, outputname):
    imwrite('outputImages/'+outputname+'.jpg', img)
    return 1

imgLoc = './hough.jpg'
rThreshold = 75;gThreshold = 0;bThreshold = 75
thresh = 50
noOfVerticalLines = 25
noOfDiagonalLines = 48 
thetaOffset = -90


def threshold(image, t):
    img = image.copy()
    for i in range(0, len(img)):
        for j in range(0, len(img[0])):
            if (img[i,j]<t):
                img[i,j] = 0
    return img


def createImage(img, Len, Wid, DiagonalL=None):
    if (DiagonalL==None):
        DiagonalL = math.ceil(math.sqrt(Len**2 + Wid**2))
    L = 2*DiagonalL+1
    W = 181
    Img = np.zeros((L, W))
    for i in range(0, Len):
        y = (Len-1)-i
        for j in range(0, Wid):
            x = j
            if(img[i,j]>thresh):
                for l in range(0, W):
                    o = math.radians(thetaOffset+l)
                    p = x*math.cos(o) + y*math.sin(o)
                    k = int( DiagonalL - math.floor(p) )
                    Img[k, l] = Img[k, l] + 1
    return Img


def findLines(baseImage, img, Len, Wid, DiagonalL=None, lineType=None):
    print(lineType+" LINES : ")
    Img = createImage(img, Len, Wid, DiagonalL)
    L, W = Img.shape
    if (lineType=='VERTICAL'):
        noOfLines = noOfVerticalLines
        for j in range(0, W):
            if (j<85 or j>95):
                Img[:, j] = 0
    elif (lineType=='DIAGONAL'):
        noOfLines = noOfDiagonalLines
        for j in range(0, W):
            if (j<=126 or j>=128):
                Img[:, j] = 0
    lines = []
    sortedImg = np.sort(Img.flatten())
    threshold_Top = sortedImg[(L*W)-noOfLines]
    for i in range(0, L):
        for j in range(0, W):
            if ( Img[i,j]>threshold_Top ):
                lines.append( ( (DiagonalL-i), (thetaOffset+j) ) )
    print("	Number of lines detected : "+str(len(lines)))
    print("	Co-ordinate Values : "+str(lines))
    rawLines = np.zeros((Len, Wid))
    for line in lines:
        p = line[0]
        o = line[1]
        o = math.radians(o)
        cosO = math.cos(o)
        sinO = math.sin(o)
        if (lineType=='VERTICAL'):
            y1 = 0
            x1 = int( (p-y1*sinO)/cosO )
            y2 = (Len-1)
            x2 = int( (p-y2*sinO)/cosO )
            l(rawLines, (x1,(Len-1)-y1),(x2,(Len-1)-y2),(255,255,255),2)
            l(baseImage, (x1,(Len-1)-y1),(x2,(Len-1)-y2),(0,0,255),2)
        else:
            y1 = 0
            x1 = int( (p-y1*sinO)/cosO )
            y2 = (Len-1)
            x2 = int( (p-y2*sinO)/cosO )
            l(rawLines, (x1,(Len-1)-y1),(x2,(Len-1)-y2),(255,255,255),2)
            l(baseImage, (x1,(Len-1)-y1),(x2,(Len-1)-y2),(255,0,0),2)
    return rawLines, baseImage


def main():
    img = imread(imgLoc)
    Len, Wid, numOfChannels = img.shape
    DiagonalL = math.ceil(math.sqrt(Len**2 + Wid**2))
    rImg = thresholdImg(img[:,:,0], rThreshold)
    gImg = thresholdImg(img[:,:,1], gThreshold)
    bImg = thresholdImg(img[:,:,2], bThreshold)
    sobelXRImg, sobelYRImg, EDRedLines = edgeDetector(rImg)
    sobelXGImg, sobelYGImg, EDGLines = edgeDetector(gImg)
    sobelXBImg, sobelYBImg, EDBlueLines = edgeDetector(bImg)
    rawLines_v, baseImage_v = findLines(img.copy(), EDRedLines, Len, Wid, DiagonalL, 'VERTICAL')
    displayImage(rawLines_v, 'BW_red_line')
    displayImage(baseImage_v, 'red_line')
    rawLines_s, baseImage_s = findLines(img.copy(), EDBlueLines, Len, Wid, DiagonalL, 'DIAGONAL')
    displayImage(rawLines_s, 'BW_blue_line')
    displayImage(baseImage_s, 'blue_line')


main()

VERTICAL LINES : 
	Number of lines detected : 24
	Co-ordinate Values : [(664, 0), (663, 0), (580, 2), (579, 2), (578, 2), (489, 3), (488, 3), (486, 2), (485, 2), (484, 2), (483, 2), (390, 2), (296, 2), (295, 2), (294, 2), (203, 3), (199, 2), (198, 2), (107, 3), (106, 3), (103, 2), (102, 2), (2, 0), (1, 0)]
DIAGONAL LINES : 
	Number of lines detected : 47
	Co-ordinate Values : [(612, 37), (611, 37), (610, 37), (609, 37), (536, 37), (535, 37), (534, 37), (533, 37), (532, 37), (531, 37), (530, 37), (466, 37), (465, 37), (464, 37), (463, 37), (462, 37), (461, 37), (460, 37), (459, 37), (458, 37), (396, 37), (395, 37), (394, 37), (393, 37), (392, 37), (391, 37), (390, 37), (389, 37), (388, 37), (324, 37), (323, 37), (322, 37), (321, 37), (320, 37), (319, 37), (318, 37), (317, 37), (316, 37), (250, 37), (249, 37), (248, 37), (247, 37), (246, 37), (245, 37), (244, 37), (243, 37), (168, 37)]
